In [ ]:
## Experimental: superceded by nestauk/wiki_topic_labels

In [ ]:
from pathlib import Path
import wikipedia
from functools import lru_cache
from collections import Counter, defaultdict
import numpy as np
import pandas as pd

@lru_cache()
def rank_wiki_labels(query):
    return Counter({result: 2**(-i) for i, result in enumerate(wikipedia.search(query))})

@lru_cache()
def get_wiki_cats(title):
    try:
        return wikipedia.page(title=title).categories
    except (wikipedia.PageError, wikipedia.DisambiguationError, wikipedia.WikipediaException):
        return Counter()
    
def accumulate_from_categories(counts, min_score=0.1):
    cumulator_terms = defaultdict(float)
    for term, score in counts.items():
        if score < min_score:
            continue
        for cat in get_wiki_cats(term):        
            if cat == term:
                continue
            if cat not in counts:
                continue
            cumulator_terms[cat] = cumulator_terms[cat] + score
    for term, count in Counter(cumulator_terms).most_common():
        counts[term] = counts[term] + 2*count
    return counts

def combos(term, min_combo=3, max_combo=5):
    terms = []
    for t in term.split():
        terms.append(t)
        n = len(terms)
        if n >= min_combo and n <= max_combo:
            yield ' '.join(terms)

In [2]:
def suggest_labels(indicator_path, supplementary_query='science', topn=3):         
    Path('suggestions').mkdir(exist_ok=True)
    topic_suggestions = []
    for path in Path(indicator_path).iterdir():
        name = path.name
        if ' '  not in name:
            continue
        counts = Counter()
        for query in combos(name):
            query = query + ' ' +supplementary_query
            ranked_labels = rank_wiki_labels(query)
            counts += ranked_labels
        counts = accumulate_from_categories(counts)
        row = dict(topic=name)
        for i, (label, _) in enumerate(counts.most_common(topn)):
            row[f'suggestion_{i}'] = label
        topic_suggestions.append(row)
    suggestions = pd.DataFrame(topic_suggestions)
    suggestions.to_csv(f'suggestions/{indicator_path}.csv')
    return suggestions

In [3]:
arxiv_suggestions = suggest_labels('arxiv')
cordis_suggestions = suggest_labels('cordis')
nih_suggestions = suggest_labels('nih')

/Users/jklinger/anaconda3/envs/py37/lib/python3.7/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /Users/jklinger/anaconda3/envs/py37/lib/python3.7/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/Users/jklinger/anaconda3/envs/py37/lib/python3.7/site-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environm